In [1]:
import mgplvm as mgp


ModuleNotFoundError: No module named 'mgplvm'

In [1]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(0)
plt.rcParams['font.size'] = 20
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False

%matplotlib

T = 19999 # number of time points
N = 252 # number of neurons
D = 2 # latent dimensionality
C = np.random.uniform(-1, 1, (N, D)) # loading matrix
sigma_n = np.random.uniform(0.2, 0.75, N) # noise parameters
ts = np.arange(T) # time series

Using matplotlib backend: Qt5Agg


In [2]:
def plot_model(X, F, Y):
  '''
  Function for plotting a draw from a generative model.
  X: latent variables (D x T)
  F: de-noised firing rates (N x T)
  Y: observations (N x T)
  Plots
  (i) the latent variables coloured according to time, 
  (ii) the latent variables coloured according to the firing rate of an example neurons, and
  (iii) the observations y_n against the firing rates f_n for two example neurons
  '''
  plt.rcParams['font.size'] = 20
  fig, axs = plt.subplots(1, 3, figsize = (12, 4))

  ### latents coloured by time ###
  axs[0].scatter(X[0, :], X[1, :], c = ts, cmap = 'coolwarm') 
  axs[0].set_title('time')

  ### latents coloured by firing rate ###
  axs[1].scatter(X[0, :], X[1, :], c = F[0, :], cmap = 'coolwarm')
  axs[1].set_title('firing rate')

  # set some labels
  for ax in axs[:2]:
    ax.set_xlabel('latent dim 1')
    ax.set_ylabel('latent dim 2')

  ### observations vs firing rate ###
  i1, i2 = np.argmax(sigma_n), np.argmin(sigma_n) #plot most and least noisy neuron
  axs[2].scatter(F[i1, :], Y[i1, :], c = 'k') 
  axs[2].scatter(F[i2, :], Y[i2, :], c = 'b')
  axs[2].legend([r'$n_1$', r'$n_2$'], frameon = False, fontsize = 16, loc = 'upper left', ncol = 2, 
                markerscale = 1.2, columnspacing = 0.15, handletextpad = 0.0005)
  # overlay an identity line for regerence
  xmin, xmax = [f(F[np.array([i1, i2]), :]) for f in [np.amin, np.amax]]
  axs[2].plot([xmin, xmax], [xmin, xmax], 'k-') 
  axs[2].set_title('observations')
  axs[2].set_xlabel('firing rate')
  axs[2].set_ylabel('recording')

  for ax in axs:
    ax.set_xticks([])
    ax.set_yticks([])

  plt.tight_layout()
  plt.show()

In [4]:
import bk.load
import bk.compute

In [6]:
bk.load.current_session_linux()
neurons,metadata = bk.load.spikes()
states = bk.load.states()
t,binned = bk.compute.binSpikes(neurons,start = 0,stop = 500)


tau = 10 # timescale of our latent process
K = np.exp( -1/(2*tau**2) * (ts[:, None] - ts[None, :])**2) # compute our RBF covariance
L = np.linalg.cholesky(K + 1e-8*np.eye(T)) # lower triangular Cholesky factor
X = np.random.normal(0, 1, (D, T)) @ L.T # x ~ N(0, LL^T) = L @ epsilon ~ N(0, I)

# F = C@X # compute firing rates as a linear function of the latents
Y = binned + np.random.normal(0, sigma_n, (T,N)).T # observations have Gaussian noise


plot_model(X, binned, Y) # plot our model draw

#### optionally plot the firing rates (F) over time as a heatmap ####
raster = True
if raster:
  print('\n\n')
  plt.figure(figsize = (10, 5))
  plt.imshow(binned, cmap = 'Greys', aspect = 'auto')
  plt.xlabel('time')
  plt.ylabel('neurons')
  plt.xticks([])
  plt.yticks([])
  plt.title('firing rates')
  plt.show()


Rat : 8 on day : 6
Working with session Rat08-20130713 @ /home/billel/Data/GG-Dataset/Rat08/Rat08-20130713
Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy


Rat : 8 on day : 6
Working with session Rat08-20130713 @ /home/billel/Data/GG-Dataset/Rat08/Rat08-20130713
Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy


In [45]:
t,binned = bk.compute.binSpikes(neurons,start = 0,stop = 500)

In [47]:
plt.imshow(binned,aspect='auto')
plt.clim([0,1])

In [46]:
binned.sum()

129549

In [51]:
plot_model(X, binned, Y) # plot our model draw


ValueError: 'c' argument has 19999 elements, which is inconsistent with 'x' and 'y' with size 100.